In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading the data
data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [ ]:
data.head(2)

In [ ]:
#Setting id as index
data.set_index('id',inplace=True)
test.set_index('id',inplace=True)

In [ ]:
data.describe()

In [ ]:
data.info()
#10K training records
#most of the input columns are continoues in nature and target is categorical

In [ ]:
data['target'].value_counts()

In [ ]:
data['target'].replace({'Class_1':0,'Class_2':1,'Class_3':2,'Class_4':3}, inplace=True)

In [ ]:
X=data.drop('target', axis=1)
y_in=data['target']
y = tf.keras.utils.to_categorical(y_in, num_classes=4, dtype='float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test  = transformer.fit_transform(X_test)
testdata= transformer.fit_transform(test)


In [ ]:
model= tf.keras.Sequential([
        
                                
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),                        
        
    
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25), 
    
    
        #tf.keras.layers.Dense(256),
        #tf.keras.layers.LeakyReLU(),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Dropout(0.25), 
    
    
        tf.keras.layers.Dense(4,activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=["acc"])

model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd=TruncatedSVD(n_components=35)
svd_inp=svd.fit_transform(X)

In [ ]:
print('\n Total Variance Explained:', round(sum(list(svd.explained_variance_ratio_))*100, 2))

In [ ]:
print('Variance of each component:', svd.explained_variance_ratio_)

In [ ]:
print('\n Total Variance Explained:', round(sum(list(svd.explained_variance_ratio_[:35]))*100, 2))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(svd_inp, y, test_size=0.33, random_state=42)

In [ ]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test  = transformer.fit_transform(X_test)
testdata= transformer.fit_transform(test)

In [ ]:
testdata=svd.fit_transform(test)

In [ ]:
model1= tf.keras.Sequential([
        
                                
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),                        
        
    
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25), 
    
    
        #tf.keras.layers.Dense(256),
        #tf.keras.layers.LeakyReLU(),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Dropout(0.25), 
    
    
        tf.keras.layers.Dense(4,activation='softmax')
])

In [ ]:
model1.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=["acc"])

model1.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
from sklearn.utils import class_weight 
import numpy as np

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(np.unique(y_in)), 
            y_in)

train_class_weights = dict(enumerate(class_weights))

In [ ]:
model2= tf.keras.Sequential([
        
                                
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25),                        
        
    
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.25), 
    
    
        #tf.keras.layers.Dense(256),
        #tf.keras.layers.LeakyReLU(),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Dropout(0.25), 
    
    
        tf.keras.layers.Dense(4,activation='softmax')
])

In [ ]:
model2.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=["acc"])

model2.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),class_weight=train_class_weights)

In [ ]:
model2.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),class_weight=train_class_weights)

In [ ]:
#best model

model1.summary()

In [ ]:
y_pred=model1.predict(testdata)

In [ ]:
predictions=[]
for row in y_pred:
    predictions.append(np.argmax(row))

In [ ]:
Id=test.index

In [ ]:
submission=pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
submission.head(2)

In [ ]:
y_pred[:,:1]

In [ ]:
df = pd.DataFrame(data = y_pred, 
                  columns = submission.columns[1:])

In [ ]:
df['id']=Id

In [ ]:
df

In [ ]:
df = df[['id', 'Class_1', 'Class_3', 'Class_2', 'Class_4']]

In [ ]:
df.head()

In [ ]:
df[submission.columns[1:]]=df[submission.columns[1:]].round(2)

In [ ]:
df.head()

In [ ]:
df.to_csv('submission.csv')